In [ ]:
import diffusion

In [ ]:
# Diffusion server connection information; same for both sessions
# adjust as needed for the server used in practice
server_url = "ws://localhost:8080"
principal = "admin"
credentials = diffusion.Credentials("password")


# handler callback function
def path_request_handler(request: str, context=None) -> str:
    return f"Hello there, {request}!"


# request properties
request = "Pushme Pullyou"
path = "path"
request_type = diffusion.datatypes.STRING  # datatype of the request

In [ ]:
# creating the two sessions
sender_session = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)
receiver_session = diffusion.Session(
    url=server_url, principal=principal, credentials=credentials
)

async with sender_session, receiver_session:
    # instantiating the messaging components for both sessions
    receiver = diffusion.Messaging(receiver_session)
    sender = diffusion.Messaging(sender_session)

    # registering the receiving request handler
    await receiver.register_request_handler(
        path,
        callback=path_request_handler,
        request_type=request_type,
        response_type=request_type,
    )

    # sending the request and receiving the response
    print(f"Sending request: '{request}'...")
    try:
        response = await sender.send_request_to_path(
            path=path, request=request_type(request)
        )
    except diffusion.DiffusionError as ex:
        print(f"ERROR: {ex}")
    else:
        print(f"... received response '{response}'")
